# Classical Crypto Algorithms

Various classical crypto algorithms and the subsequent frequency cryptanalysis on them. Source for a lot of understanding was Implementing Cryptography S. Bray.

### An abstract class for different ciphers

In [2]:
from abc import ABC, abstractmethod

class Cipher(ABC):
    ''' An abstract base class that forces and encrypt and decrypt interface.'''
    
    def __init__(self,):
        ''' Initialisation'''
        super().__init__()
        
    @abstractmethod
    def encrypt(self):
        pass

    @abstractmethod
    def decrypt(self):
        pass

### Substitution Cipher

Simple substition cipher e.g. Caesar, Rot-N.

In [20]:
class SubstitionCipher(Cipher):
    ''' Simple substition cipher that shifts the alphabet by a given rotation.'''

    def __init__(self, nShift):
        
        # Amount to shift the substition by.
        self.n = nShift

        
    def encrypt(self, plainText, key):
        ''' Encoder function to convert a plain text into an encrypted cipher text.
            This function will convert the input to lower case.
        INPUTS: 
            plainText - a text string.
            key - a string (that must be all lower case).
        OUTPUTS:
            cipherText - the encrypted text string.
        '''
        
        # Create an empty string to store the cipher text.
        cipherText = ''
        # For each letter in the plain text convert it to lower case (as the key )
        for l in plainText.lower():
            try:
                i = (key.index(l) + self.n) % 26
                cipherText += key[i]

            except ValueError:
                cipherText += l

        return cipherText

    
    def decrypt(self, cipherText, key):
        ''' Encoder function to convert a plain text into an encrypted cipher text.
            This function will convert the input to lower case.
        INPUTS: 
            cipherText - the encrypted text string.
            key - a string (that must be all lower case).
        OUTPUTS:
            msg - the decrypted text string.
        '''  

        # Create an empty string to store the decrypted message.
        msg = ''
        for l in cipherText:
            try:
                i = (key.index(l) - self.n) % 26
                msg += key[i]
            except ValueError:
                msg += l

        return msg


In [21]:
key = 'abcdefghijklmnopqrstuvwxyz'
msg = 'Once more unto the breach'
nShift = 6 # NB If this is 13 then same function can act as encoder and decoder due to symmetry.
subCycle = SubstitionCipher(nShift)
cipherText = subCycle.encrypt( msg, key)
print(cipherText)
decodeMsg = subCycle.decrypt(cipherText, key)
print(decodeMsg)

utik suxk atzu znk hxkgin
once more unto the breach


### Vigenere Cipher

This Cipher is another substition Cipher. Instead of a single substition it uses all of the 26 possible Caesar ciphers. It uses the key to select which of the 26 shifts to select.

In [238]:
class Vigenere(Cipher):
    ''' Vigenere cipher that uses a keyword to specify which of 26 rotations should be used.'''

    def __init__(self):
        ''' Initialise the class.'''
        
        # Possible letters in alphabet.
        self.alphabetPossible =  {'A':0, 'B':1, 'C':2, 'D':3, 
                            'E':4, 'F':5, 'G':6,  'H':7,  'I':8,
                            'J':9,  'K':10,'L':11,  'M':12, 'N': 13,
                            'O':14,  'P':15, 'Q':16, 'R':17, 
                            'S':18, 'T':19, 'U':20, 'V':21, 'W':22,
                            'X':23,'Y':24, 'Z':25, }
        
        self.lenAlphabet = len(self.alphabetPossible)
        
    
    def keyShiftIdx(self, key):
        ''' Convert the key word to it's corresponding index in the alphabet.'''
        
        # First make sure key is in uppercase.
        key = key.upper()
        
        # Use the numerical index of the letters in the key (i.e. A=0, D=3), to create
        # a new keyArray. The new keyArray contains just the indexes and will be used 
        # in the Vigenere cypher as the amount of shift to rotate the alphabet by.
        keyArray = []
        for i in range(0, len(key)):
            keyElement = ord(key[i]) - ord('A') # subtract the start of upper case letters to base at 0.
            keyArray.append(keyElement)
            
        return keyArray
 
    
    def shift(self, character, n):
        ''' Encode the character by shifting it by n places in the alphabet.
        INPUT:
            character - The character to shift/rotate.
            n - The number of places to shift the character by.
        OUTPUT:
            The shifted character (converted to lower case letters).
        '''

        # Shift the capitalised character by n units. If the shift goes over 
        # the end of the alphabet apply the modulus operator to ensure it wraps.
        # NB the substraction of ord('A') is to make sure the modulus operator
        # is acting on a number between 0 and 26.
        shiftedInt = (ord(character) - ord('A') + n) % self.lenAlphabet

        # Due to subtracting ord('A') we need to add an offset back if we want 
        # the cipher text to be in the range of usual letters. We can either add
        # ord('A') to create capitalised letters or ord('a') for lower case. 
        # Adding ord('a') here.
        shiftedInt += ord('a')

        # Convert to a character.
        shiftedChar = chr(shiftedInt)

        return shiftedChar

    
    def encrypt(self, plainTxt, key):
        ''' Encrypt a plain text with a supplied key.
        INPUTS: 
            plainText - a text string.
            key - a string (that must be all lower case).
        OUTPUTS:
            cipherText - the encrypted text string.
        '''
        
        # Check the text is all in upper characters.
        plainTxt = plainTxt.upper()
        
        # Strip any unwanted characters i.e. not in the alphabet.
        plainTxtSafe = "".join([l for l in plainTxt if l in self.alphabetPossible])

        
        print(plainTxtSafe)
        
        # Now create the cipher text. This is done by taking each character in the message and
        # shifting by the number specified in the key. 
        cipherTxt = "".join([self.shift(plainTxtSafe[i], key[i % len(key)]) for i in range(len(plainTxtSafe))])
        
        return cipherTxt
    
    
    def decrypt(self, cipherTxt, key):
        ''' Decrypt a cipher text with a supplied key.
        INPUTS: 
            cipherTxt - a text string.
            key - a string (that must be all lower case).
        OUTPUTS:
            plainTxt - the decrypted text string.
        '''
        
        # Convert cipher text to upper case (in agreement with alphabet).
        cipherTxtUpper = cipherTxt.upper()
                
        # Same as encrypt but have a negative shift of the key to go backwards.
        plainTxt = "".join([self.shift(cipherTxtUpper[i], -key[i % len(key)]) for i in range(len(cipherTxtUpper))])
        
        return plainTxt.upper()

In [244]:
# Create the Vigenere cipher.
vig = Vigenere()

# Create a secret keyword that is only known by the sender and recipient.
keyWord = 'testWordThatShouldBeSecure'

# Convert the keyword to the indices of the letters (e.g. A=0, B=1 etc).
key = vig.keyShiftIdx(keyWord)

# Define a message.
msg = "HELLO - VIGENERE DO YOU WORK"
print(msg)

# Encrypt the message.
cipherTxt = vig.encrypt(msg, key)
print("The encrypted message is: {}".format(cipherTxt))

# Now decrypt the message.
msgDecrypt = vig.decrypt(cipherTxt, key)
print("The decrypted message is: {}".format(msgDecrypt))


HELLO - VIGENERE DO YOU WORK
HELLOVIGENEREDOYOUWORK
The encrypted message is: aidekjzjxuekwkcszxxsjo
The decrypted message is: HELLOVIGENEREDOYOUWORK


### Affine Cipher

A substition cipher that uses the modulo function. For encryption letters are converted to integer e.g. A=00, C=02, W=22, then modular function converts this number to another number which is then substituted into the ciphertext (as the corresponding letter). The decrypt process uses the inverse modulo.   

In [245]:
class AffineCipher(Cipher):
    ''' Use modulo function (ax+b) mod m to encrypt. m represents the length of the alphabet, and x is the letter (as an int) to encrypt.
        a and b are two ints used in the key.
    '''
    
    def __init__(self):
        pass
    
    
    def encrypt(self, plainText, a, b, m):
        """
            Implement cipherText = a*plainText+b mod m.
        """
    
        # Create a placeholder to store the cipher.
        cipherText = ''
        
        # Iterate through the message and apply modulo formula to integer representation.
        for t in plainText.upper().replace(' ', ''):
            
            # Apply formula
            mod = ((key[0]* (ord(t) - ord('A')) + key[1] )  % m) + ord('A')  # NB subtract by ord('A') as start of alphabet. Add ord('A') to convert back to upper.
            
            # Convert back to a character and store in ciphertext.
            cipherText +=  chr(mod)  
    
        return cipherText

    
    def euclidGCD(self, a, b):
        ''' Get the greatest common divisor- Taken from Implementing Cryptography.'''
        
        x = 0
        y = 1
        u = 1
        v = 0
        
        while a != 0:
            q = b//a
            r = b%a
            m = x-u*q
            n = y-v*q
            
            b,a, x,y, u,v = a,r, u,v, m,n
            
        gcd = b
        
        return gcd, x, y
    
    def inverseMod(self, a, m):
        ''' Taken from implementing Cryptography.'''
        
        gcd, x, y = self.euclidGCD(a,m)
        if gcd != 1:
            return -1 # Failure.
        else:
            return x % m
    
    def decrypt(self, cipherText, a, b, m):
        ''' Implement the modulo inverse.'''
        
        # Create a placeholder for the plain text.
        plainText = ''
        
        # Iterate through the cipher text and apply the inverse modulo formula.
        for t in cipherText:
            
            # Get inverse modulo.
            temp =  (self.inverseMod(a, m)*(ord(t) - ord('A') - b) ) %m + ord('A')
                        
            # Convert back to a character and store in plain text.
            plainText += chr(temp)
    
        return plainText

In [248]:
# Test the affine cipher out.
affCipher = AffineCipher()
msg = 'once more unto the breach'
print(msg)

lenAlphabet = 26
key = [17,20]

# Encrypt.
cipherText = affCipher.encrypt(msg, key[0], key[1], lenAlphabet)
print(cipherText)

# Decrypt.
decryptText = affCipher.decrypt(cipherText, key[0], key[1], lenAlphabet)
print(decryptText)

once more unto the breach
YHCKQYXKWHFYFJKLXKUCJ
ONCEMOREUNTOTHEBREACH
